In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd

from bokeh.models import LinearAxis, Range1d, LabelSet, ColumnDataSource
from bokeh.layouts import gridplot, row

In [4]:
from math import sqrt
import numpy as np

In [5]:
from bokeh.io import export_png

In [6]:
%ls

1510419211.csv        setup_0.05_erws/      test-test.csv
bak/                  setup_1.0/            test.csv
result.csv            setup_1.0_erws/       visualization.ipynb
setup_0.05/           test-lb-policy.csv    visualization2.ipynb


# 1. Constrained optimization

## 1.1 Short setup delay

In [7]:
df=pd.read_csv('setup_0.05/output_1.csv', skiprows=1)

if df.iloc[-1][0] > 0:
    df = df.iloc[-1:]
else:
    df = df.iloc[1:2]

for i in range(2,45):
    df_tmp = pd.read_csv('setup_0.05/output_'+str(i)+'.csv', skiprows=1)
    if df_tmp.iloc[-1][0] > 0:
        df_tmp = df_tmp.iloc[-1:]
    else:
        df_tmp = df_tmp.iloc[1:2]
    df = df.append(df_tmp)

df['relative_idle_time'] = df.small_arrival * df.avg_idle_time
#df.relative_idle_time    = df.relative_idle_time.apply(lambda x: round(x))

#df_rnd.relative_idle_time    = df_rnd.relative_idle_time.apply(lambda x: round(x))

In [8]:
def logscale(arrival):
    rel_idle = df[df.small_arrival==arrival].relative_idle_time
    rel_idle = rel_idle.apply(lambda x: np.log2(x)).tolist()
    rel_idle.pop(0)
    rel_idle.insert(0, -10)
    
    return rel_idle

In [9]:
p0 = figure(plot_width=600, plot_height=400)


p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time, color='white', line_color='navy',size=5)



p0.line([-10,0,1,2,10],0.1*np.ones(5),line_color='red',legend='constraint')

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1
p0.outline_line_color = 'black'

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. inter-arrival time)'
p0.yaxis.axis_label = 'Mean response time'

p0.xaxis.axis_label_text_color = 'black'
p0.xaxis.major_label_text_color = 'black'
p0.yaxis.axis_label_text_color = 'black'
p0.yaxis.major_label_text_color = 'black'
p0.legend.label_text_color = 'black'

p0.axis.axis_label_text_font_size = '12pt'
p0.axis.major_label_text_font_size = '10pt'

p0.legend.location  = 'top_right'

show(p0)

In [10]:
p0 = figure(plot_width=600, plot_height=400)


p0.line(logscale(1), df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. inter-arrival time)'
p0.yaxis.axis_label = 'Mean power'

p0.xaxis.axis_label_text_color = 'black'
p0.xaxis.major_label_text_color = 'black'
p0.yaxis.axis_label_text_color = 'black'
p0.yaxis.major_label_text_color = 'black'
p0.legend.label_text_color = 'black'

p0.legend.location  = 'bottom_right'
p0.outline_line_color = 'black'

p0.axis.axis_label_text_font_size = '12pt'
p0.axis.major_label_text_font_size = '10pt'

show(p0)

## 1.2 Long setup delay

In [11]:
%ls

1510419211.csv        setup_0.05_erws/      test-test.csv
bak/                  setup_1.0/            test.csv
result.csv            setup_1.0_erws/       visualization.ipynb
setup_0.05/           test-lb-policy.csv    visualization2.ipynb


In [12]:
df=pd.read_csv('setup_1.0/output_1.csv', skiprows=1)

if df.iloc[-1][0] > 0:
    df = df.iloc[-1:]
else:
    df = df.iloc[1:2]

for i in range(2,45):
    df_tmp = pd.read_csv('setup_1.0/output_'+str(i)+'.csv', skiprows=1)

    if df_tmp.iloc[-1][0] > 0:
        df_tmp = df_tmp.iloc[-1:]
    else:
        df_tmp = df_tmp.iloc[1:2]
    df = df.append(df_tmp)

df['relative_idle_time'] = df.small_arrival * df.avg_idle_time

In [13]:
p0 = figure(plot_width=600, plot_height=400)

#source = ColumnDataSource(df.loc[0:27])

p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time.tolist(), line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time, color='white', line_color='navy',size=5)



p0.line([-10, 0,1,2,10],0.6*np.ones(5),line_color='red',legend='constraint')

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. inter-arrival time)'
p0.yaxis.axis_label = 'Mean response time'

p0.xaxis.axis_label_text_color = 'black'
p0.xaxis.major_label_text_color = 'black'
p0.yaxis.axis_label_text_color = 'black'
p0.yaxis.major_label_text_color = 'black'
p0.legend.label_text_color = 'black'

p0.legend.location  = 'top_right'
p0.outline_line_color = 'black'

p0.axis.axis_label_text_font_size = '12pt'
p0.axis.major_label_text_font_size = '10pt'

show(p0)

In [14]:
p0 = figure(plot_width=600, plot_height=400)


p0.line(logscale(1), df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. inter-arrival time)'
p0.yaxis.axis_label = 'Mean power'

p0.xaxis.axis_label_text_color = 'black'
p0.xaxis.major_label_text_color = 'black'
p0.yaxis.axis_label_text_color = 'black'
p0.yaxis.major_label_text_color = 'black'
p0.legend.label_text_color = 'black'

p0.legend.location  = 'top_right'
p0.outline_line_color = 'black'

p0.axis.axis_label_text_font_size = '12pt'
p0.axis.major_label_text_font_size = '10pt'
#export_png(p0, filename='./power-vs-idle-logscale.png')


show(p0)

# 2. Weighted sum cost

In [15]:
%ls 

1510419211.csv        setup_0.05_erws/      test-test.csv
bak/                  setup_1.0/            test.csv
result.csv            setup_1.0_erws/       visualization.ipynb
setup_0.05/           test-lb-policy.csv    visualization2.ipynb


In [16]:
%ls setup_1.0_erws/

output_1.csv     output_20.csv    output_31.csv    output_42.csv
output_10.csv    output_21.csv    output_32.csv    output_43.csv
output_11.csv    output_22.csv    output_33.csv    output_44.csv
output_12.csv    output_23.csv    output_34.csv    output_5.csv
output_13.csv    output_24.csv    output_35.csv    output_6.csv
output_14.csv    output_25.csv    output_36.csv    output_7.csv
output_15.csv    output_26.csv    output_37.csv    output_8.csv
output_16.csv    output_27.csv    output_38.csv    output_9.csv
output_17.csv    output_28.csv    output_39.csv    output_None.csv
output_18.csv    output_29.csv    output_4.csv
output_19.csv    output_3.csv     output_40.csv
output_2.csv     output_30.csv    output_41.csv


## 2.1 Short setup delay

In [17]:
df=pd.read_csv('setup_0.05_erws/output_1.csv', skiprows=1)

if df.iloc[-1][0] > 0:
    df = df.iloc[-1:]
else:
    df = df.iloc[1:2]

for i in range(2,45):
    df_tmp = pd.read_csv('setup_0.05_erws/output_'+str(i)+'.csv', skiprows=1)
    #df_rnd_tmp = df_tmp.loc[0,:]
    #df_rnd = df_rnd.append(df_rnd_tmp)
    if df_tmp.iloc[-1][0] > 0:
        df_tmp = df_tmp.iloc[-1:]
    else:
        df_tmp = df_tmp.iloc[1:2]
    df = df.append(df_tmp)

df['relative_idle_time'] = df.small_arrival * df.avg_idle_time

p0 = figure(plot_width=600, plot_height=400)


p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time + 0.01 * df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time + 0.01 * df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time + 0.01 * df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time + 0.01 * df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time + 0.01 * df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time + 0.01 * df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time + 0.01 * df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time + 0.01 * df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. inter-arrival time)'
p0.yaxis.axis_label = 'Weighted sum cost'

p0.xaxis.axis_label_text_color = 'black'
p0.xaxis.major_label_text_color = 'black'
p0.yaxis.axis_label_text_color = 'black'
p0.yaxis.major_label_text_color = 'black'
p0.legend.label_text_color = 'black'

p0.legend.location  = 'bottom_right'
p0.outline_line_color = 'black'

p0.axis.axis_label_text_font_size = '12pt'
p0.axis.major_label_text_font_size = '10pt'
#export_png(p0, filename='./weighted-sum-short-setup.png')

show(p0)

## 2.2 long setup delay

In [18]:
df=pd.read_csv('setup_1.0_erws/output_1.csv', skiprows=1)

if df.iloc[-1][0] > 0:
    df = df.iloc[-1:]
else:
    df = df.iloc[1:2]

for i in range(2,45):
    df_tmp = pd.read_csv('setup_1.0_erws/output_'+str(i)+'.csv', skiprows=1)
    #df_rnd_tmp = df_tmp.loc[0,:]
    #df_rnd = df_rnd.append(df_rnd_tmp)
    if df_tmp.iloc[-1][0] > 0:
        df_tmp = df_tmp.iloc[-1:]
    else:
        df_tmp = df_tmp.iloc[1:2]
    df = df.append(df_tmp)

df['relative_idle_time'] = df.small_arrival * df.avg_idle_time

p0 = figure(plot_width=600, plot_height=400)


p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time + 0.01 * df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time + 0.01 * df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time + 0.01 * df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time + 0.01 * df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time + 0.01 * df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time + 0.01 * df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time + 0.01 * df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time + 0.01 * df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. inter-arrival time)'
p0.yaxis.axis_label = 'Weighted sum cost'

p0.xaxis.axis_label_text_color = 'black'
p0.xaxis.axis_line_color = 'black'
p0.xaxis.major_label_text_color = 'black'
p0.xaxis.major_tick_line_color = 'black'
p0.xaxis.minor_tick_line_color = 'black'

p0.yaxis.axis_label_text_color = 'black'
p0.yaxis.axis_line_color = 'black'
p0.yaxis.major_label_text_color = 'black'
p0.yaxis.major_tick_line_color = 'black'
p0.yaxis.minor_tick_line_color = 'black'

p0.legend.label_text_color = 'black'
p0.legend.background_fill_alpha = 0.6
p0.legend.background_fill_color = 'orange'
p0.legend.location  = 'top_right'
p0.outline_line_color = 'black'

p0.axis.axis_label_text_font_size = '12pt'
p0.axis.major_label_text_font_size = '10pt'
#export_png(p0, filename='./weighted-sum-long-setup.png')

show(p0)